In [1]:
import json
import svgling
from svgling.figure import Caption, SideBySide, RowByRow
from IRVVisualisationUtils import treeListToTuple, parseAssertions, printAssertions, buildRemainingTreeAsLists, buildPrintedResults, printTrees

a_file = open("../AssertionJSON/SanFran_2007.json")
auditfile = json.load(a_file)
        
(apparentWinner, apparentNonWinners, WOLosers,IRVElims) = parseAssertions(auditfile)
                
elimTrees = buildPrintedResults(apparentWinner, apparentNonWinners, WOLosers,IRVElims)

print("Built "+str(len(elimTrees))+" trees to visualise excluded alternate winners.")



Apparent winner: 7
Apparently eliminated: ['15', '14', '13', '16', '12', '17', '5', '0', '3', '1', '8', '10', '2', '4', '6', '9', '11']


Built 17 trees to visualise excluded alternate winners.


# RAIRE example assertion parser and visualizer

This notebook parses and visualizes RAIRE assertions.
Right now it's hardcoded to read RAIRE_sample_audit1.json, but you can change that.
Start by executing the rectangle above to understand the election and the apparent winner.
The audit needs to exclude all the other possible winners, though we don't care about other elimination orders in which the apparent winner still wins.
Execute the next code snippet to see the trees of possible alternative elimination orders.
Each tree will be pruned according to RAIRE's assertions, with each pruned branch tagged with the assertion that allowed us to exclude it.
You (the auditor) need to check that all the leaves end in an assertion, which shows that they have been excluded.


In [7]:
Caption(printTrees(elimTrees),"Trees showing how other winners are excluded.")


NameError: name 'printTrees' is not defined

Now print all the assertions.  This gives you an explanation of the meaning of each one.


In [3]:
printAssertions(WOLosers,IRVElims)

Not-Eliminated-Before assertions: 
NEB  0: Candidates {'7'} cannot be eliminated before 6.
NEB  1: Candidates {'7'} cannot be eliminated before 4.
NEB  2: Candidates {'7'} cannot be eliminated before 2.
NEB  3: Candidates {'7'} cannot be eliminated before 11.
NEB  4: Candidates {'7'} cannot be eliminated before 9.
NEB  5: Candidates {'7'} cannot be eliminated before 17.
NEB  6: Candidates {'7'} cannot be eliminated before 0.
NEB  7: Candidates {'7'} cannot be eliminated before 1.
NEB  8: Candidates {'7'} cannot be eliminated before 5.
NEB  9: Candidates {'7'} cannot be eliminated before 8.
NEB 10: Candidates {'7'} cannot be eliminated before 10.
NEB 11: Candidates {'7'} cannot be eliminated before 12.
NEB 12: Candidates {'7'} cannot be eliminated before 13.
NEB 13: Candidates {'7'} cannot be eliminated before 14.
NEB 14: Candidates {'7'} cannot be eliminated before 15.
NEB 15: Candidates {'7'} cannot be eliminated before 16.
NEB 16: Candidates {'7'} cannot be eliminated before 3.


Not

Now the audit begins! We now apply a Risk Limiting Audit to test each of the assertions above.
For each assertion, we consider the opposite hypothesis, that candidate C *can* be eliminated at that point. We then try to audit until that hypothesis can be rejected.  If all the hypotheses are rejected, the election result is declared correct.  At any time, if the audit has failed to reject all the hypotheses, a full manual recount can be conducted.
